In [1]:
from pymongo import MongoClient

In [2]:
import pickle

In [3]:
from pprint import pprint

In [4]:
client = MongoClient('mongodb://localhost:27017/')

In [5]:
client.list_database_names()

['admin', 'config', 'local', 'vacancies_db']

In [6]:
vac_db = client.vacancies_db

Подтягиваем данные из предыдущего урока

In [63]:
with open('vacancies_rabota_ru', 'rb') as f:
    vacancies_rabota_ru = pickle.load(f)

In [64]:
with open('vacancies_hh_ru', 'rb') as f:
    vacancies_hh_ru = pickle.load(f)

In [65]:
vac_db.hh_ru.drop()
vac_db.hh_ru.insert_many(vacancies_hh_ru)

In [66]:
vac_db.rabota_ru.drop()
vac_db.rabota_ru.insert_many(vacancies_rabota_ru)

### Задание №1

In [10]:
def add_new_vacancy(vacancy, db_collection):
    try:
        db_collection.find({'ref':vacancy['ref']})[0]
        print(f'Такая вакансия уже присутствует в базе данных.')
    except IndexError:
        db_collection.insert_one(vacancy)
        print('Вакансия успешно добавлена!')

In [11]:
add_new_vacancy({
    'platform': 'nn.rabota.ru',
     'ref': 'https://nn.rabota.ru/vacancy/vacancy/35485411/?search_id=1663520004703349itfwf7',
     'salary': {'comment': None,
                'currency': 'руб.',
                'max': '45000',
                'min': '35000'},
     'salary_raw': '35\xa0000\xa0— 45\xa0000 руб.',
     'title': 'Бухгалтер'}, vac_db.rabota_ru)

Такая вакансия уже присутствует в базе данных.


### Задание №2

In [12]:
def find_vacancies_salary_greater_than(db_collection, amount):
    collection = db_collection.find({'$or':[{'salary.min':{'$gt':amount}}, {'salary.max':{'$gt':amount}}]})
    results_list = []
    for vac in collection:
        results_list.append(vac)
    return results_list

In [70]:
vacancies_list = find_vacancies_salary_greater_than(vac_db.rabota_ru, '60000')

In [71]:
vacancies_list

[{'_id': ObjectId('632b469effb26a9941742566'),
  'ref': 'https://nn.rabota.ru/vacancy/vacancy/46933422/?search_id=16637804521585ivztu3se1l',
  'salary_raw': '50\xa0000\xa0— 70\xa0000 руб.',
  'salary': {'min': '50000',
   'max': '70000',
   'currency': 'руб.',
   'comment': None},
  'title': 'Программист 1с',
  'platform': 'nn.rabota.ru',
  'employer': 'Jobers',
  'clear_title': 'программист1с',
  'clear_employer': 'jobers'},
 {'_id': ObjectId('632b469effb26a9941742595'),
  'ref': 'https://nn.rabota.ru/vacancy/vacancy/35009028/',
  'salary_raw': '35\xa0000\xa0— 70\xa0000 руб.',
  'salary': {'min': '35000',
   'max': '70000',
   'currency': 'руб.',
   'comment': None},
  'title': 'Сервисный инженер IT',
  'platform': 'nn.rabota.ru',
  'employer': 'АСТ, ООО (Нижний Новгород)',
  'clear_title': 'сервисныйинженерit',
  'clear_employer': 'астооо(нижнийновгород)'},
 {'_id': ObjectId('632b469effb26a9941742597'),
  'ref': 'https://nn.rabota.ru/vacancy/vacancy/46867169/',
  'salary_raw': 'от 90

### Задание №3

In [58]:
def match_vacancies(collection_1, collection_2):
    result = []
    for vacancy_1 in collection_1.find():
        matches = collection_2.find({'clear_title':vacancy_1['clear_title']})
        for match in matches:
            if vacancy_1['clear_employer'] == match['clear_employer']:
                result.append((vacancy_1, match))
            elif (vacancy_1['clear_employer'] is None) or (vacancy_1['clear_employer'] is None):
                if vacancy_1['salary']['min'] == match['salary']['min'] and vacancy_1['salary']['max'] == match['salary']['max']:
                    result.append((vacancy_1, match))
    return result

In [67]:
results = match_vacancies(vac_db.rabota_ru, vac_db.hh_ru)

In [68]:
pprint(results)

[({'_id': ObjectId('632b469effb26a9941742564'),
   'clear_employer': '2гис',
   'clear_title': 'seniorgolangdeveloper',
   'employer': '2ГИС',
   'platform': 'nn.rabota.ru',
   'ref': 'https://nn.rabota.ru/vacancy/vacancy/46974931/?search_id=16637804521585ivztu3se1l',
   'salary': {'comment': 'по договоренности',
              'currency': None,
              'max': None,
              'min': None},
   'salary_raw': 'по договоренности',
   'title': 'Senior Golang Developer'},
  {'_id': ObjectId('632b469effb26a9941742540'),
   'clear_employer': '2гис',
   'clear_title': 'seniorgolangdeveloper',
   'employer': '2ГИС',
   'platform': 'nn.hh.ru',
   'ref': 'https://nn.hh.ru/vacancy/68135135?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
   'salary': {'comment': None, 'currency': None, 'max': None, 'min': None},
   'salary_raw': None,
   'title': 'Senior Golang Developer'}),
 ({'_id': ObjectId('632b469effb26a9941742566'),
   'clear_employer': 'jobers',
   'clear_t